<a href="#Overview"></a>
# Overview
* <a href="#2fa87a0d-1aac-421f-8311-ca19a9cba02b">NEUS 642 - Week 2 In-class exercises</a>
  * <a href="#44967bb5-d77d-484f-a06b-016baaf8aba7">Note about formating functions</a>
* <a href="#9e97fa5a-21c2-47cb-8456-f6e5805e90dd">Starting conditions, from homework</a>
* <a href="#e64feaf5-2c46-4ee3-a3b2-9e2dd3fed122">Plot a tuning curve with error bars</a>
  * <a href="#ab132ffe-be0c-4a5b-8a04-4d83e2611349">Question 1 - Function to compute spontaneous rate.</a>
  * <a href="#f344b2e4-7f7b-4fc6-a8a6-161fbd8a5660">Question 2 - Function to compute mean and SEM response for each tone</a>
  * <a href="#346b570b-4f0b-4328-9de1-c27dfa35abc5">Question 3 - Make a pretty plot with error bars</a>
* <a href="#8131403b-adea-4e63-80d3-85296ed3d8a5">Bonus question</a>

<a id="2fa87a0d-1aac-421f-8311-ca19a9cba02b"></a>
# NEUS 642 - Week 2 In-class exercises
<a href="#Overview">Return to overview</a>


<a id="44967bb5-d77d-484f-a06b-016baaf8aba7"></a>
## Note about formating functions
<a href="#Overview">Return to overview</a>

If you're given an empty, placeholder function (aka, a "stub"), please use this for formatting your answer. This should help you organize your thoughts. 

Dummy question. Write a function that takes two inputs, a and b, and returns their sum:


In [ ]:
def add_numbers(a,b):
    # Your work here

    return c

In [ ]:
# Your answer here

In [ ]:
# test it -- will not work until you add the correct code inside the function
add_numbers(1,2)

<a id="9e97fa5a-21c2-47cb-8456-f6e5805e90dd"></a>
# Starting conditions, from homework
<a href="#Overview">Return to overview</a>


We'll continue working with the spike data from class. So first, let's initialize this instance of python and load the datasets, same as in the lecture notebook. Refer to the lecture notebook if you need to remind yourself about the formatting of the data.

In [ ]:
import numpy as np

In [ ]:
spikes1 = np.loadtxt('data/CRD016c-40-1.csv', delimiter=',')
cell1_label = 'CRD016c-40-1'
spikes2 = np.loadtxt('data/CRD016c-50-1.csv', delimiter=',')
cell2_label = 'CRD016c-50-1'

stim = np.loadtxt('data/stim.csv', delimiter=',')
stim = np.round(stim, 3)
print("Loaded spike times for two neurons and list of stimulus events")
print("spikes1 - number of spike events:", spikes1.shape)
print("spikes2 - number of spike events:", spikes2.shape)
print("stim    - number of stimuli X [freq, start, stop]", stim.shape)

<a id="e64feaf5-2c46-4ee3-a3b2-9e2dd3fed122"></a>
# Plot a tuning curve with error bars
<a href="#Overview">Return to overview</a>

The frequency tuning curve we generated in class plots the mean spike rate response to each distinct stimulus, but it doesn't indicate which of those *responses are signficantly different from the spontaneous spike rate*. We want to determine which stimulus frequencies evoke a response.

**Statistics primer:** One quick way to assess significance is to measure the standard error on the mean (SEM), which is the standard deviation across a list of samples $(x)$, divided by the square root of the number of samples. Important detail for computing statistics on real data: estimating standard deviation and SEM requires a "degree of freedom" (DOF) correction, which means dividing by $n-1$ instead of $n$ (The basic intuition for this correction is that you need one DOF in your n data points to estimate the mean, $m$, so you only have $n-1$ left to compute variance/SEM):

$SEM = \frac{1}{\sqrt{n-1}} \sqrt{(\sum_i^n{(x(i)-m)^2} / (n-1))} = \frac{1}{n-1} \sqrt{(\sum_i^n{(x(i)-m)^2}}$

As a simple rule of thumb, the measurement of a mean value is signficant with $p<0.05$ if it is at least **2 SEM** from baseline. 

**Another note/warning(!!)** SEM works if your data samples are *independent and normally distributed*. Because we randomly varied the order of stimulus presentation, we can pretty safely assume the samples are independent. "Normal" means the distribution can be described by a Gaussian function. Spiking activity isn't normal, but let's assume it is today. We'll learn more about non-parametric statistics, which are better for analyzing spiking and other biological data later.

But for now, let's stick with SEM. Using `numpy`, we can compute SEM with the `np.std` function. Use the `ddof` parameter to include the degrees-of-freedom correction.

So we will do three things:

1. Compute the spontaneuous spike rate for a neuron as a measure of baseline.
2. Compute the mean and SEM response to each tone (include 1-degree of freedom correction)
3. Plot the mean response to each tone with error bars to indicate whether 2 SEM do or do not overlap with the spont rate.

A bit more about SEM calculation.

In [ ]:
x = np.array([4,4,4,5,5,5,6,6,6])
n = len(x)
std = np.std(x, ddof=1)
sem = std / np.sqrt(n-1)
print("SEM for", len(x), "samples:", sem)

Note that as we collect more samples, our SEM typically gets smaller, meaning we're more confident about our measurment of the mean. Standard deviation does not get smaller.

In [ ]:
x2 = np.array([4,4,4,5,5,5,6,6,6,4,5,6,4,5,6,4,5,6])
n=len(x2)
sem = np.std(x2, ddof=1)/np.sqrt(n-1)
print("SEM for", len(x2), "samples:", sem)

There are many ways to skin a cat. You can compute SEM directly without the `std` function. And/or use the alternative formulation of `np.sqrt(x)` as an exponent, `x**0.5`. 

In [ ]:
m = np.mean(x2)
sem = np.sum((x2-m)**2)**0.5/(n-1)
print("Same thing:", sem)

<a id="ab132ffe-be0c-4a5b-8a04-4d83e2611349"></a>
## Question 1 - Function to compute spontaneous rate.
<a href="#Overview">Return to overview</a>

It's considered good practice to break down, or *modularize*, analysis code into small chunks, making it easier to debug and reuse. One particular place where modularization is helpful is between code that performs computations and code that plots the results.

So let's start with something simple and bite-sized. To assess if a response is significant, we need to measure the change in spike rate relative to the spontaneous spike rate in units of spikes/sec. Let's write a function that calculates spont rate from our data. This should be easy, simply "packaging" code from the lecture inside a function like this:

```
spont_rate = calc_spont_rate(stim, spikes)
```

**Note** that the variable containing the spike data is called `spikes` (not `spikes1` or `spikes2`).  Why is that?  We also pass the `stim` variable to this function. Why is that imporant?

In [ ]:
# Hard-coding a 0.1 second pre-stimulus window for measuring spontaneous activity
pre_duration = 0.1

def calc_spont_rate(stim, spikes):
    """
    Average spike rates across the pre-stimulus window for all stimulis 
    listed in stim and return the average spont rate in units of spikes/sec.
    """
    # Your work here
    
    return spont_rate

In [ ]:
# Your answer here

Test it. This should produce the same answer as we got during lecture.

In [ ]:
calc_spont_rate(stim, spikes1)

And we can call the same function for the second neuron:

In [ ]:
calc_spont_rate(stim, spikes2)

<a id="f344b2e4-7f7b-4fc6-a8a6-161fbd8a5660"></a>
## Question 2 - Function to compute mean and SEM response for each tone
<a href="#Overview">Return to overview</a>

Next step: write a function that takes our `spikes` and `stim` arrays, and returns three arrays: a list of unique stimulus frequencies, the mean response to each frequency and the SEM for each frequency. Again, you should be able to borrow a most of the code from the lecture notebook.

Syntax:
```
f, m, sem = tuning_mean_sem(stim, spikes)

```
Notice, the template specifies that this function should return a tuple of THREE vectors.

In [ ]:
def tuning_mean_sem(stim, spikes):
    """
    Determine the list of unique stimulus frequencies and then compute 
    the mean and SEM spike rate for each frequencies (as in class, computed 
    across the time period from stimulus onset to offset).
    """
    # Your answer here

    return f, m, sem

In [ ]:
# Your answer here

Test it with data from the first unit (`spikes1`):

In [ ]:
f, m, sem = tuning_mean_sem(stim, spikes1)

i = 6
print(f"For {f[i]} Hz stimulus, mean response is {m[i]:.3f} spikes/sec, and SEM is {sem[i]:.3f} spikes/sec")

In [ ]:
# find the response to the stimulus closest to 1000 Hz
flookup = 1000
fmatch = np.argmin(np.abs(f-flookup))
print(f"For {f[fmatch]} Hz stimulus, mean response is {m[fmatch]:.3f} spikes/sec, and SEM is {sem[fmatch]:.3f} spikes/sec")

<a id="346b570b-4f0b-4328-9de1-c27dfa35abc5"></a>
## Question 3 - Make a pretty plot with error bars
<a href="#Overview">Return to overview</a>

Finally: using the results generated by your two functions, write a function that generates a tuning curve with error bars to indicate stimuli that produce a significant response. The error bars in your plot should indicate **2 SEMs**. Points where the error bars do not cross the spont line are signficant responses.

The function `plt.errorbar` should come in handy. Important syntax detail: `errorbar` requires three inputs. `errorbar(x,y,e)`, where x indicates values on the x axis, in this case, frequency.

Don't forget to label your x and y axes with accurate and interpretable units. And use `cell_label` to give your plot a title. Syntax:
```
plot_mean_sem(f, m, sem, cell_label)
```

In [ ]:
# We'll use matplotlib to generate the figure
import matplotlib.pyplot as plt


In [ ]:
def plot_mean_sem(f, m, sem, cell_label):
    # Your answer here
    
    return

In [ ]:
# Your answer here

Test your code:

In [ ]:
spont = calc_spont_rate(stim, spikes1)
f, m, sem = tuning_mean_sem(stim, spikes1)

plot_mean_sem(f, m, sem, cell1_label)

Now let's enjoy the satisfaction of nice modular code that can be applied to data from a second neuron:

In [ ]:
spont = calc_spont_rate(stim, spikes2)
f, m, sem = tuning_mean_sem(stim, spikes2)

plot_mean_sem(f, m, sem, cell2_label)

<a id="8131403b-adea-4e63-80d3-85296ed3d8a5"></a>
# Bonus question
<a href="#Overview">Return to overview</a>

Look pretty! But interesting, are the error bars convincing? Remember the point about assuming a normal distribution from above. Any thoughts on why/how the error bars may not be accurate? Don't worry if you don't have ideas. This is a subtle but important aspect of data analysis that will come up again.

In [ ]:
# Your thoughts here.